In [1]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.tsatools import add_trend, add_lag

In [2]:
products_df = pd.read_csv("./tbl_products.txt")
products_df['periodo'] = pd.to_datetime(products_df['periodo'])
products_df = products_df[products_df["periodo"] <= "2019-02"]
# data = data[data["product_id"] == 20001]
products_df = products_df.drop(['Unnamed: 0'], axis=1)
products_df

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,cat1,cat2,cat3,brand,sku_size
0,2017-01-01,10234,20524,2,0.05300,HC,VAJILLA,Cristalino,Importado,500
1,2017-01-01,10032,20524,1,0.13628,HC,VAJILLA,Cristalino,Importado,500
2,2017-01-01,10217,20524,1,0.03028,HC,VAJILLA,Cristalino,Importado,500
3,2017-01-01,10125,20524,1,0.02271,HC,VAJILLA,Cristalino,Importado,500
4,2017-01-01,10012,20524,11,1.54452,HC,VAJILLA,Cristalino,Importado,500
...,...,...,...,...,...,...,...,...,...,...
2903530,2019-02-01,10205,20032,1,0.36973,HC,ROPA LAVADO,Liquido,ROPEX2,3000
2903531,2019-02-01,10375,20032,1,0.05602,HC,ROPA LAVADO,Liquido,ROPEX2,3000
2903532,2019-02-01,10217,20032,1,0.86270,HC,ROPA LAVADO,Liquido,ROPEX2,3000
2903533,2019-02-01,10043,20032,1,0.86270,HC,ROPA LAVADO,Liquido,ROPEX2,3000


In [3]:
data = products_df.groupby(by=['product_id', 'periodo', 'cat1'], group_keys=False).sum("cust_request_tn")

In [4]:
data_grouped = products_df.groupby(by=['product_id', 'periodo', 'cat1'])[['cust_request_tn']].describe()
dataFrameData = pd.DataFrame(data_grouped)
dataFrameData

cust_request_tn                                 \
                                      count       mean        std      min   
product_id periodo    cat1                                                   
20001      2017-01-01 HC              186.0   5.041544  20.310446  0.01119   
           2017-02-01 HC              185.0   4.506605  18.638272  0.01119   
           2017-03-01 HC              188.0   7.078441  30.911744  0.02239   
           2017-04-01 HC              104.0  10.893695  40.636599  0.01119   
           2017-05-01 HC              238.0   6.515502  25.257114  0.01119   
...                                     ...        ...        ...      ...   
21271      2019-02-01 REF               6.0   0.001735   0.001945  0.00037   
21282      2017-01-01 PC                9.0   0.005431   0.004152  0.00163   
           2017-02-01 PC                7.0   0.009776   0.005722  0.00326   
21287      2017-01-01 FOODS             3.0   0.011067   0.008639  0.00415   
21294      2017-08-01 PC                1.0   0.007640        NaN  0.00764   

                                                                     
                                  25%      50%       75%        max  
product_id periodo    cat1                                           
20001      2017-01-01 HC     0.111930  0.22386  0.968197  184.72927  
           2017-02-01 HC     0.111930  0.24625  1.108110  198.84365  
           2017-03-01 HC     0.111930  0.24625  0.805893  313.90769  
           2017-04-01 HC     0.123120  0.30221  1.231230  267.35600  
           2017-05-01 HC     0.123120  0.24625  0.984980  216.36069  
...                               ...      ...       ...        ...  
21271      2019-02-01 REF    0.000750  0.00093  0.001673    0.00557  
21282      2017-01-01 PC     0.003260  0.00326  0.006520    0.01303  
           2017-02-01 PC     0.005705  0.00977  0.013035    0.01792  
21287      2017-01-01 FOODS  0.006225  0.00830  0.014525    0.02075  
21294      2017-08-01 PC     0.007640  0.00764  0.007640    0.00764  

[21781 rows x 8 columns]

Agrego estadisticos basandos en las toneldas (cust_request_tn)

In [5]:
data["tn_count"] = dataFrameData["cust_request_tn"]["count"]
data["tn_mean"] = dataFrameData["cust_request_tn"]["mean"]
data["tn_std"] = dataFrameData["cust_request_tn"]["std"]
data["tn_min"] = dataFrameData["cust_request_tn"]["min"]
data["tn_25%"] = dataFrameData["cust_request_tn"]["25%"]
data["tn_50%"] = dataFrameData["cust_request_tn"]["50%"]
data["tn_75%"] = dataFrameData["cust_request_tn"]["75%"]
data["tn_max"] = dataFrameData["cust_request_tn"]["max"]
data

customer_id  cust_request_qty  cust_request_tn  \
product_id periodo    cat1                                                    
20001      2017-01-01 HC         1893102               479        937.72717   
           2017-02-01 HC         1881286               432        833.72187   
           2017-03-01 HC         1912750               509       1330.74697   
           2017-04-01 HC         1055299               279       1132.94430   
           2017-05-01 HC         2423431               701       1550.68936   
...                                  ...               ...              ...   
21271      2019-02-01 REF          60477                 6          0.01041   
21282      2017-01-01 PC           91180                19          0.04888   
           2017-02-01 PC           70757                18          0.06843   
21287      2017-01-01 FOODS        30176                 6          0.03320   
21294      2017-08-01 PC           10003                 1          0.00764   

                             sku_size  tn_count    tn_mean     tn_std  \
product_id periodo    cat1                                              
20001      2017-01-01 HC       558000     186.0   5.041544  20.310446   
           2017-02-01 HC       555000     185.0   4.506605  18.638272   
           2017-03-01 HC       564000     188.0   7.078441  30.911744   
           2017-04-01 HC       312000     104.0  10.893695  40.636599   
           2017-05-01 HC       714000     238.0   6.515502  25.257114   
...                               ...       ...        ...        ...   
21271      2019-02-01 REF         120       6.0   0.001735   0.001945   
21282      2017-01-01 PC         1350       9.0   0.005431   0.004152   
           2017-02-01 PC         1050       7.0   0.009776   0.005722   
21287      2017-01-01 FOODS        30       3.0   0.011067   0.008639   
21294      2017-08-01 PC          350       1.0   0.007640        NaN   

                              tn_min    tn_25%   tn_50%    tn_75%     tn_max  
product_id periodo    cat1                                                    
20001      2017-01-01 HC     0.01119  0.111930  0.22386  0.968197  184.72927  
           2017-02-01 HC     0.01119  0.111930  0.24625  1.108110  198.84365  
           2017-03-01 HC     0.02239  0.111930  0.24625  0.805893  313.90769  
           2017-04-01 HC     0.01119  0.123120  0.30221  1.231230  267.35600  
           2017-05-01 HC     0.01119  0.123120  0.24625  0.984980  216.36069  
...                              ...       ...      ...       ...        ...  
21271      2019-02-01 REF    0.00037  0.000750  0.00093  0.001673    0.00557  
21282      2017-01-01 PC     0.00163  0.003260  0.00326  0.006520    0.01303  
           2017-02-01 PC     0.00326  0.005705  0.00977  0.013035    0.01792  
21287      2017-01-01 FOODS  0.00415  0.006225  0.00830  0.014525    0.02075  
21294      2017-08-01 PC     0.00764  0.007640  0.00764  0.007640    0.00764  

[21781 rows x 12 columns]

In [6]:
data.reset_index(inplace=True)

In [7]:
data

,product_id,periodo,cat1,customer_id,cust_request_qty,cust_request_tn,sku_size,tn_count,tn_mean,tn_std,tn_min,tn_25%,tn_50%,tn_75%,tn_max
0,20001,2017-01-01,HC,1893102,479,937.72717,558000,186.0,5.041544,20.310446,0.01119,0.111930,0.22386,0.968197,184.72927
1,20001,2017-02-01,HC,1881286,432,833.72187,555000,185.0,4.506605,18.638272,0.01119,0.111930,0.24625,1.108110,198.84365
2,20001,2017-03-01,HC,1912750,509,1330.74697,564000,188.0,7.078441,30.911744,0.02239,0.111930,0.24625,0.805893,313.90769
3,20001,2017-04-01,HC,1055299,279,1132.94430,312000,104.0,10.893695,40.636599,0.01119,0.123120,0.30221,1.231230,267.35600
4,20001,2017-05-01,HC,2423431,701,1550.68936,714000,238.0,6.515502,25.257114,0.01119,0.123120,0.24625,0.984980,216.36069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21776,21271,2019-02-01,REF,60477,6,0.01041,120,6.0,0.001735,0.001945,0.00037,0.000750,0.00093,0.001673,0.00557
21777,21282,2017-01-01,PC,91180,19,0.04888,1350,9.0,0.005431,0.004152,0.00163,0.003260,0.00326,0.006520,0.01303
21778,21282,2017-02-01,PC,70757,18,0.06843,1050,7.0,0.009776,0.005722,0.00326,0.005705,0.00977,0.013035,0.01792
21779,21287,2017-01-01,FOODS,30176,6,0.03320,30,3.0,0.011067,0.008639,0.00415,0.006225,0.00830,0.014525,0.02075


In [8]:
for lag in range(10):
    newLag = lag + 1
    data['lag' + str(newLag)] = data['cust_request_tn'].shift(newLag).apply(lambda x: 0 if pd.isna(x) else x)


In [9]:
df_trends = add_trend(data[["cust_request_tn"]], trend='ctt')
df_trends

,cust_request_tn,const,trend,trend_squared
0,937.72717,1.0,1.0,1.0
1,833.72187,1.0,2.0,4.0
2,1330.74697,1.0,3.0,9.0
3,1132.94430,1.0,4.0,16.0
4,1550.68936,1.0,5.0,25.0
...,...,...,...,...
21776,0.01041,1.0,21777.0,474237729.0
21777,0.04888,1.0,21778.0,474281284.0
21778,0.06843,1.0,21779.0,474324841.0
21779,0.03320,1.0,21780.0,474368400.0


In [10]:
# data = data.drop(['Unnamed: 0'], axis=1)
# data.describe().T

In [11]:

# lag 1-15 (listo)
# estadisticos (listo)
# tendencias 2-7 (meses)
# ventas 1 anio atras
# ventas 10 meses atras
# estacionalidad (modulo del mes entre 100)


In [12]:
data

,product_id,periodo,cat1,customer_id,cust_request_qty,cust_request_tn,sku_size,tn_count,tn_mean,tn_std,...,lag1,lag2,lag3,lag4,lag5,lag6,lag7,lag8,lag9,lag10
0,20001,2017-01-01,HC,1893102,479,937.72717,558000,186.0,5.041544,20.310446,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,20001,2017-02-01,HC,1881286,432,833.72187,555000,185.0,4.506605,18.638272,...,937.72717,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,20001,2017-03-01,HC,1912750,509,1330.74697,564000,188.0,7.078441,30.911744,...,833.72187,937.72717,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,20001,2017-04-01,HC,1055299,279,1132.94430,312000,104.0,10.893695,40.636599,...,1330.74697,833.72187,937.72717,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,20001,2017-05-01,HC,2423431,701,1550.68936,714000,238.0,6.515502,25.257114,...,1132.94430,1330.74697,833.72187,937.72717,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21776,21271,2019-02-01,REF,60477,6,0.01041,120,6.0,0.001735,0.001945,...,0.00745,0.00668,0.04456,0.07204,0.07019,0.04792,0.02785,0.01671,0.03194,0.03603
21777,21282,2017-01-01,PC,91180,19,0.04888,1350,9.0,0.005431,0.004152,...,0.01041,0.00745,0.00668,0.04456,0.07204,0.07019,0.04792,0.02785,0.01671,0.03194
21778,21282,2017-02-01,PC,70757,18,0.06843,1050,7.0,0.009776,0.005722,...,0.04888,0.01041,0.00745,0.00668,0.04456,0.07204,0.07019,0.04792,0.02785,0.01671
21779,21287,2017-01-01,FOODS,30176,6,0.03320,30,3.0,0.011067,0.008639,...,0.06843,0.04888,0.01041,0.00745,0.00668,0.04456,0.07204,0.07019,0.04792,0.02785


In [14]:
data['tn_std'] = data['tn_std'].fillna(0)

In [16]:
data

,product_id,periodo,cat1,customer_id,cust_request_qty,cust_request_tn,sku_size,tn_count,tn_mean,tn_std,...,lag1,lag2,lag3,lag4,lag5,lag6,lag7,lag8,lag9,lag10
0,20001,2017-01-01,HC,1893102,479,937.72717,558000,186.0,5.041544,20.310446,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,20001,2017-02-01,HC,1881286,432,833.72187,555000,185.0,4.506605,18.638272,...,937.72717,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,20001,2017-03-01,HC,1912750,509,1330.74697,564000,188.0,7.078441,30.911744,...,833.72187,937.72717,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,20001,2017-04-01,HC,1055299,279,1132.94430,312000,104.0,10.893695,40.636599,...,1330.74697,833.72187,937.72717,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,20001,2017-05-01,HC,2423431,701,1550.68936,714000,238.0,6.515502,25.257114,...,1132.94430,1330.74697,833.72187,937.72717,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21776,21271,2019-02-01,REF,60477,6,0.01041,120,6.0,0.001735,0.001945,...,0.00745,0.00668,0.04456,0.07204,0.07019,0.04792,0.02785,0.01671,0.03194,0.03603
21777,21282,2017-01-01,PC,91180,19,0.04888,1350,9.0,0.005431,0.004152,...,0.01041,0.00745,0.00668,0.04456,0.07204,0.07019,0.04792,0.02785,0.01671,0.03194
21778,21282,2017-02-01,PC,70757,18,0.06843,1050,7.0,0.009776,0.005722,...,0.04888,0.01041,0.00745,0.00668,0.04456,0.07204,0.07019,0.04792,0.02785,0.01671
21779,21287,2017-01-01,FOODS,30176,6,0.03320,30,3.0,0.011067,0.008639,...,0.06843,0.04888,0.01041,0.00745,0.00668,0.04456,0.07204,0.07019,0.04792,0.02785


In [17]:
data.to_csv(
    path_or_buf="products_with_lag_and_descriptions.txt"
)